In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from citipy import citipy
import requests as req
import unidecode
import time
from math import sqrt
from datetime import datetime

In [ ]:
#keys
from keys import (gkey, wkey)

In [ ]:
# Build data frame of randomly generated lat and long
location_data = pd.DataFrame()
location_data['rand_lat'] = [np.random.uniform(-90,90) for x in range(1500)]
location_data['rand_lng'] = [np.random.uniform(-180, 180) for x in range(1500)]

# add closest city and country column
location_data['closest_city'] = ""
location_data['country'] = ""

#find and add closest city and country code
for index, row in location_data.iterrows():
    lat = row['rand_lat']
    lng = row['rand_lng']
    location_data.set_value(index, 'closest_city', citipy.nearest_city(lat, lng).city_name)
    location_data.set_value(index, 'country', citipy.nearest_city(lat, lng).country_code)

In [ ]:
# delete repeated cities and find unique city count
location_data = location_data.drop_duplicates(['closest_city', 'country'])
location_data = location_data.dropna()
len(location_data['closest_city'].value_counts())

In [ ]:
# USE BELOW VALUE FOR COUNT MATCHING 
rec_check = len(location_data['closest_city'])  #Difference is because some city names occur in different countries.
rec_check

In [ ]:
#preview data
location_data.head()

In [ ]:
# keep only city and country
# Remove random lats and lngs (no longer needed)
location_data = location_data[['closest_city', 'country']]

#rename column headers
location_data = location_data.rename(columns = {'closest_city': 'city'})
location_data.head()

In [ ]:
# read in open weather map's country Id json
# downloaded from https://openweathermap.org/appid#work per Openweathermaps documentation

api_city_data = pd.read_json('city.list.json')

for index, row in api_city_data.iterrows():
    lower_city = row['name'].lower() #make all city name lowercase
    unaccented = unidecode.unidecode(lower_city) # strip accents from city name
    lower_country = row['country'].lower() # make all two digit county 
    api_city_data.set_value(index, 'name', unaccented) # reset the value of name (city) to stripped down version
    api_city_data.set_value(index, 'country', lower_country) # reset the value of country to lower case
    
api_city_data = api_city_data.rename(columns = {'name': 'city'}) # rename for merge

In [ ]:
#left merge with random cities from location_data

merged_df = location_data.merge(api_city_data, how = 'left', on = ('city', 'country'))
merged_df = merged_df.drop_duplicates(['city', 'country']) #drop duplicates

# to verify with number below
len(merged_df)

In [ ]:
#check with above
rec_check

In [ ]:
#preview merged_df
merged_df.head()

In [ ]:
#data types clean-up
merged_df['coord'] = merged_df['coord'].fillna('') #fill na cells with empty string for coordinates
merged_df['id'] = merged_df['id'].fillna(0) # fill na with 0 for id in order to change to int64
merged_df['id'] = merged_df['id'].astype(dtype = 'int64') # cast id column as type int64 to remove floating .0
merged_df['id'].dtype #check type of id

In [ ]:
#check how many returned valid ids
#merged_df['id'].value_counts()

In [ ]:
# check which countries did not find ids
no_id = merged_df[merged_df['id'] == 0]
no_id.head()

In [ ]:
#check how many without ids
len(no_id)

In [ ]:
#trying to find lat and lng for cities missing ids from google geocoding api
g_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='

counter = 0 #for check of all cities
for index,row in merged_df.iterrows():
    if row['id'] == 0:
        city = row['city']
        country = row['country']
        print('Now retrieving coordinates for city #%s: %s, %s' %(index, city, country))
        target_url = '%s%s,+%s&key=%s' % (g_url, city, country, gkey)
        print(target_url)
        try:
            response = req.get(target_url).json()
            response_path = response['results'][0]['geometry']['location']
            merged_df.set_value(index, 'coord', {'lon': response_path['lng'], 'lat': response_path['lat']})
        except:
            print('Missing Data for city #%s: %s,%s' %(index, city, country))
        counter += 1
#     if counter == 10:
#             break

print(counter) #to check for same number of records as no_id

In [ ]:
#preview merged_df
merged_df.head(30)

In [ ]:
#check with below
len(merged_df)

In [ ]:
#check with above
rec_check

In [ ]:
#check to see how many records with no coordinates
no_coord = merged_df[merged_df['coord'] == ""]
no_coord

In [ ]:
# of records without coordinates
len(no_coord)

In [ ]:
#leave merged_df the same from here on
weather_data = merged_df.copy()

In [ ]:
# Open Weather Maps Api example links for reference
# id url example: 'api.openweathermap.org/data/2.5/weather?id=2172797'
# coord url example: 'api.openweathermap.org/data/2.5/weather?lat=35&lon=139'
# city search use : api.openweathermap.org/data/2.5/weather?q=London,uk
# remember to use &appid= for key

counter = 0 #for breaking and pausing
cur_err_list = [] # for cities without data from current weather
for_err_list = [] # for cities without data from forecast data
cur_errors = 0  #current weather pull errors
for_errors = 0 #forecast weather pull errors

#create additional columns for open weather map data

#make columnds for lat and lng from open weather source
weather_data['lat'] = ""
weather_data['lng'] = ""

#make columns for current weather data (at time of pull)
weather_data['cur_date'] = ""
weather_data['cur_temp'] = ""
weather_data['cur_humidity'] = ""
weather_data['cur_clouds'] = ""
weather_data['cur_wind'] = ""

# make columns for records corresponding to the highest temperature 
# forecasted in the next 24 hours (from time of pull)
weather_data['max_date'] = ""
weather_data['max_temp'] = ""
weather_data['max_temp_humidity'] = ""
weather_data['max_temp_clouds'] = ""
weather_data['max_temp_wind'] = ""

# make columns for records corresponding to the average values
# forecasted in the next 5 days (from time of pull)
weather_data['avg_date0'] = ""
weather_data['avg_date1'] = ""
weather_data['avg_temp'] = ""
weather_data['avg_humidity'] = ""
weather_data['avg_clouds'] = ""
weather_data['avg_wind'] = ""

t0 = time.time() #for pause timer
for index, row in weather_data.iterrows():
    print('Now retrieving data for city #%s: %s, %s' % (index, row['city'], row['country']))
    #uses url believed to be most accurate
    if ((row['id']) == 0) and (row['coord'] != ""): # coordinates if no id, but with coordinates
        lat = row['coord']['lat']
        lon = row['coord']['lon']
        cur_url = 'https://api.openweathermap.org/data/2.5/weather?lat=%s&lon=%s&APPID=%s&units=imperial' % (lat, lon, wkey)  
        for_url = 'https://api.openweathermap.org/data/2.5/forecast?lat=%s&lon=%s&APPID=%s&units=imperial' % (lat, lon, wkey)  
    elif row['id'] != 0: # use if if ID exists
        loc_id = row['id']
        cur_url = 'https://api.openweathermap.org/data/2.5/weather?id=%s&APPID=%s&units=imperial' % (loc_id, wkey)
        for_url = 'https://api.openweathermap.org/data/2.5/forecast?id=%s&APPID=%s&units=imperial' % (loc_id, wkey)
    else: #use city and country if no id AND no coordinates
        city = row['city']
        country = row['country']
        cur_url = 'https://api.openweathermap.org/data/2.5/weather?q=%s,%s&APPID=%s&units=imperial' % (city, country, wkey)
        for_url = 'https://api.openweathermap.org/data/2.5/forecast?q=%s,%s&APPID=%s&units=imperial' % (city, country, wkey)
    print('Current Weather URL:')
    print(cur_url)
    print('Forecast Weather URL:')
    print(for_url)
    #get current weather data
    try:
        cur_response = req.get(cur_url).json()
        weather_data.set_value(index, 'lat', cur_response['coord']['lat'])
        weather_data.set_value(index, 'lng', cur_response['coord']['lon'])
        weather_data.set_value(index, 'cur_date', cur_response['dt'])
        weather_data.set_value(index, 'cur_temp', cur_response['main']['temp'])
        weather_data.set_value(index, 'cur_humidity', cur_response['main']['humidity'])
        weather_data.set_value(index, 'cur_clouds', cur_response['clouds']['all'])
        weather_data.set_value(index, 'cur_wind', cur_response['wind']['speed'])
    except:
        print('Missing Current Weather Info for city #%s: %s, %s' % (index, row['city'], row['country']))
        cur_err_list.append(index)
        cur_errors += 1
    try:
        #get max temperature weather data
        for_response = req.get(for_url).json()
        for_path = for_response['list']
        temps_24h = [] # a list of temp over a 24 hours period forecased every 3 hours
        for n in range(9): # a 24 hour period
            temps_24h.append(for_path[n]['main']['temp_max'])
        max_index = temps_24h.index(max(temps_24h))
        weather_data.set_value(index, 'max_date', for_path[max_index]['dt'])
        weather_data.set_value(index, 'max_temp', for_path[max_index]['main']['temp_max'])
        weather_data.set_value(index, 'max_temp_humidity', for_path[max_index]['main']['humidity'])
        weather_data.set_value(index, 'max_temp_clouds', for_path[max_index]['clouds']['all'])
        weather_data.set_value(index, 'max_temp_wind', for_path[max_index]['wind']['speed'])
        # get avg forecast values 
        #set up blank lists for dates, temperature, clouds, wind, humidity over a 5 day period
        dat = []
        tem = []
        clo = []
        win = []
        hum = []
        for n in for_path: # 5 days worth of forecast
            dat.append(n['dt'])
            tem.append(n['main']['temp'])
            clo.append(n['clouds']['all'])
            win.append(n['wind']['speed'])
            hum.append(n['main']['humidity'])
        weather_data.set_value(index, 'avg_date0', dat[0]) #beginning date
        weather_data.set_value(index, 'avg_date1', dat[-1]) #ending date
        weather_data.set_value(index, 'avg_temp', np.mean(tem)) # mean temp over 5 days
        weather_data.set_value(index, 'avg_humidity', np.mean(hum)) #mean humidity over 5 days
        weather_data.set_value(index, 'avg_clouds', np.mean(clo)) #mean cloud cover over 5 days
        weather_data.set_value(index, 'avg_wind', np.mean(win)) #mean wind speed over 5 days
    except:
        print('Missing Forecast Info for city #%s: %s, %s' % (index, row['city'], row['country']))
        for_err_list.append(index)
        for_errors += 1
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    counter +=1
    if counter % 30 == 0: #because two records pulled for each city 
        t1 = time.time() #records time very 30 records
        sl_time = 70 - (t1-t0) # calculates buffer for api pull limit
        print("")
        print('********Sleeping for %s seconds.********' % (sl_time))
        print("")
        time.sleep(sl_time) # pauses for appropraite amount of time
        t0 = time.time() # resets for next 30 pull timer
#     if counter == 5:
#         break

In [ ]:
# Summary Report
print("Counts and Errors Report")
print('---------------------------------------------------------------------------')
print("Count")
print('----------------')
print(str(counter))
print('---------------------------------------------------------------------------')
print("Errors")
print('----------------')
print('# of Current Weather Errors: ' + str(cur_errors))
print("Current Weather Errors Index List:")

if len(cur_err_list) > 0:
    for n in cur_errors:
        print(n)
else:
    print('None')
print("")
print('# of Forecast Weather Errors:' + str(for_errors))
print("Forecast Weather Errors Index List: ")
if len(for_err_list) > 0:
    for n in for_errors:
        print(n)
else:
    print('None')
print('---------------------------------------------------------------------------')

In [ ]:
#view columns
weather_data.columns

In [ ]:
# clean up
clean_col = weather_data.columns[4:]
clean_col

In [ ]:
#loop throught to clean columns to be able to use for graphs
for c in clean_col:
    weather_data[c] = pd.to_numeric(weather_data[c], errors = 'coerce') 
    weather_data = weather_data[weather_data[c].isnull() == False]

len(weather_data)

In [ ]:
# check weather data types
weather_data.dtypes

In [ ]:
#Export to csv
weather_data.to_csv('Data_Output/clean_weather_data.csv')

In [ ]:
########## Data CSV read to avoid pull requests again  ########
# weather_data = pd.read_csv('Data_Output/clean_weather_data.csv')
# weather_data.head()

In [ ]:
#date sorting and conversions dictionary for graph labels

dates = {'max_cur': weather_data['cur_date'].max(),
         'min_cur': weather_data['cur_date'].min(),
         'max_max': weather_data['max_date'].max(),
         'min_max': weather_data['max_date'].min(),
         'min_avg': weather_data['avg_date0'].max(),
         'max_avg': weather_data['avg_date1'].min()
        }

for key in dates.keys():
    convert = datetime.utcfromtimestamp(dates[key]).strftime('%Y-%m-%d%, %I:%M:%S %p')
    dates[key] = convert

In [ ]:
# dictionary for labels
labels_dic = {"cur_temp": "Current Temperature", 
              'max_temp': 'Maximum Temp 24 Hours', 
              'avg_temp': 'Average Forecasted Temp in 5 Days',
             'cur_humidity': 'Current Humidity',
             'max_temp_humidity': "Forecasted Humidity (%) at the Maximum Temperature in 24 Hours",
             'avg_humidity': 'Average Forecasted Humidity (%) over 5 Days',
             'cur_clouds': 'Current Cloud Cover (%)',
             'max_temp_clouds': 'Forecasted Cloud Cover (%) at the Maximum Forecasted Temperature in 24 Hours',
             'avg_clouds': 'Average Forecasted Cloud Cover (%) over 5 Days',
             'cur_wind': 'Current Wind Speed (mph)',
             'max_temp_wind': 'Forecasted Wind Speed (mph) at the Maximum Forecasted Temperature in 24 Hours',
             'avg_wind': 'Average Forecasted Wind Speed (mph) over 5 Days'}

In [ ]:
# Temp vs Latitude Graphs
temp_list = ['cur_temp', 'max_temp', 'avg_temp']  #would have done only dict but wanted consistent order.

xvals = weather_data['lat']

for temp in temp_list:
    # y values of each item in list for separate graphs
    yvals = weather_data[temp]
    #adds title including title and timestamp range of sample data
    plt.title("%s vs Latitude \n Samples Taken from %s to %s UTC" % (labels_dic[temp], dates['min_' + temp.split('_')[0]],  dates['max_' + temp.split('_')[0]]))
    plt.axvline(0, color = 'black', alpha = .25, label = 'Equator') #adds equator line
    plt.text(1,30,'Equator',rotation=90)
    plt.ylim(15, 120) #to give consistent scale
    plt.xlabel('Latitude')
    plt.ylabel("Temperature (F)")
    plt.scatter(xvals, yvals)
    plt.show()
    plt.savefig("Graph_Output/%s vs Latitude.png" % (labels_dic[temp]))

In [ ]:
# Humidity vs Latitude Graphs
#see first set of graphs commenting for notes
hum_list = ['cur_humidity', 'max_temp_humidity', 'avg_humidity']  #would have done only dict but wanted consistent order.

xvals = weather_data['lat']

for hum in hum_list:
    yvals = weather_data[hum]
    plt.title("%s vs Latitude \n Samples Taken from %s to %s UTC" % (labels_dic[hum], dates['min_' + hum.split('_')[0]],  dates['max_' + hum.split('_')[0]]))
    plt.xlabel('Latitude')
    plt.ylabel('Humidity (%)')
    plt.axvline(0, color = 'black', alpha = .25, label = 'Equator')
    plt.text(1,20,'Equator',rotation=90)
    plt.scatter(xvals, yvals)
    plt.show()
    plt.savefig("Graph_Output/%s vs Latitude.png" % (labels_dic[hum]))

In [ ]:
# Cloud Cover vs Latitude Graphs
#see first set of graphs commenting for notes
cloud_list = ['cur_clouds', 'max_temp_clouds', 'avg_clouds']  #would have done only dict but wanted consistent order.

xvals = weather_data['lat']

for clo in cloud_list:
    yvals = weather_data[clo]
    plt.title("%s vs Latitude \n Samples Taken from %s to %s UTC" % (labels_dic[clo], dates['min_' + clo.split('_')[0]],  dates['max_' + clo.split('_')[0]]))
    plt.xlabel('Latitude')
    plt.ylabel('Cloud Cover (%)')
    plt.ylim(-5,105)
    plt.axvline(0, color = 'black', alpha = .25, label = 'Equator')
    plt.text(-5,-20,'Equator')
    plt.scatter(xvals, yvals)
    plt.show()
    plt.savefig("Graph_Output/%s vs Latitude.png" % (labels_dic[clo]))

In [ ]:
# Wind Speed vs Latitude Graphs
#see first set of graphs commenting for notes
win_list = ['cur_wind', 'max_temp_wind', 'avg_wind']  #would have done only dict but wanted consistent order.

xvals = weather_data['lat']

for win in win_list:
    yvals = weather_data[win]
    plt.title("%s vs Latitude \n Samples Taken from %s to %s UTC" % (labels_dic[win], dates['min_' + win.split('_')[0]],  dates['max_' + win.split('_')[0]]))
    plt.xlabel('Latitude')
    plt.ylabel('Wind Speed (mph))')
    plt.ylim(-5,60)
    plt.axvline(0, color = 'black', alpha = .25, label = 'Equator')
    plt.text(1,35,'Equator',rotation=90)
    plt.scatter(xvals, yvals)
    plt.show()
    plt.savefig("Graph_Output/%s vs Latitude.png" % (labels_dic[win]))

In [ ]:
#graphs lats vs long, temperature scale from red(hot) to green(cool), and bubble size based on humidity, cloud cover, and wind speed
xvars = weather_data['lng']
yvars = weather_data['lat']
color = weather_data['avg_temp']
size_list = ['avg_humidity', 'avg_clouds', 'avg_wind']

#loops through size list and only changes size of bubbles based on different variables
for measure in size_list:  
    plt.figure(figsize = (18,12))
    plt.xlim(-180,180)
    plt.ylim(-90,90)
    plt.title("Global Temperatures Based on 5 Day Average \n Samples taken from %s to %s UTC \n Note:  Bubble Size Corresponds to %s" % (dates['min_avg'], dates['max_avg'], labels_dic[measure]))
    plt.axhline(0, color = 'black', alpha = .25, label = 'Equator')
    plt.text(-155,3,'Equator')
    size = weather_data[measure]
    plt.scatter(xvars, 
                yvars, 
                c = color, 
                s = size * 6, 
                edgecolor = 'black', 
                linewidth = 1, 
                alpha = .5, 
                cmap=plt.cm.RdYlGn_r)
    plt.show()
    plt.savefig("Graph_Output/Global Temperatures Based on 5 Day Average Bubble Plot: %s.png" % (measure))